

*Original code by Ryan S. McGee. Modified by T.W. Alleman in consultation with the BIOMATH research unit headed by prof. Ingmar Nopens.*

Copyright (c) 2020 by T.W. Alleman, BIOMATH, Ghent University. All Rights Reserved.

This notebook accompanies our preprint: "*A deterministic, age-stratified, extended SEIRD model for assessing the effect of non-pharmaceutical interventions on SARS-CoV-2 spread in Belgium*"(https://doi.org/10.1101/2020.07.17.20156034)

# Load required packages

In [1]:
import random
import os
import numpy as np
import json
import corner
import random

import pandas as pd
import datetime
import scipy
import matplotlib.dates as mdates
import matplotlib
import math
import xarray as xr
import emcee
import matplotlib.pyplot as plt
import datetime

from covid19model.optimization import objective_fcns,pso
from covid19model.models import models
from covid19model.models.utils import draw_sample_COVID19_SEIRD_google
from covid19model.models.time_dependant_parameter_fncs import google_lockdown, ramp_fun, contact_matrix
from covid19model.data import google
from covid19model.data import sciensano
from covid19model.data import model_parameters
from covid19model.visualization.output import population_status, infected, _apply_tick_locator 
from covid19model.visualization.optimization import plot_fit, traceplot


# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Load data

In [2]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(dataset='willem_2012')
levels = initN.size
Nc_all = {'total': Nc_total, 'home':Nc_home, 'work': Nc_work, 'schools': Nc_schools, 'transport': Nc_transport, 'leisure': Nc_leisure, 'others': Nc_others}


In [3]:
df_sciensano = sciensano.get_sciensano_COVID19_data(update=False)
df_sciensano.tail(2)

,H_tot,ICU_tot,H_in,H_out,H_tot_cumsum,D_tot,D_25_44,D_45_64,D_65_74,D_75_84,D_85+
DATE,,,,,,,,,,,
2020-11-24,4820,1105,286,528,4594,83,0.0,12.0,10,24,37
2020-11-25,4570,1071,272,429,4437,17,1.0,4.0,3,3,6


In [4]:
df_google = google.get_google_mobility_data(update=False, plot=False)
df_google.tail(2)

,retail_recreation,grocery,parks,transport,work,residential
date,,,,,,
2020-11-21,-58.0,-11.0,13.0,-41.0,-20.0,13.0
2020-11-22,-63.0,-18.0,-11.0,-45.0,-21.0,10.0


In [ ]:

# def switch_beta(t,param,samples_dict):
#     if t < pd.to_datetime('2020-05-04'):
#         return np.random.choice(samples_dict['beta'],1,replace=False)
#     elif pd.to_datetime('2020-05-04') < t <= pd.to_datetime('2020-09-01'):
#         return np.random.choice(samples_dict['beta_summer'],1,replace=False)
#     else:
#         return np.random.choice(samples_dict['beta'],1,replace=False)

# Wave 1 - March 2020 - June 2020

In [5]:
def wave1_policies(t,param,df_google, Nc_all, l , tau, 
                   prev_schools, prev_work, prev_transport, prev_leisure, prev_others, prev_home):
    
    # Convert tau and l to dates
    tau_days = pd.Timedelta(tau, unit='D')
    l_days = pd.Timedelta(l, unit='D')

    # Define additional dates where intensity or school policy changes
    t1 = pd.Timestamp('2020-03-15') # start of lockdown
    t2 = pd.Timestamp('2020-05-18') # gradual re-opening of schools (15%)
    t3 = pd.Timestamp('2020-06-04') # further re-opening of schools (65%)
    t4 = pd.Timestamp('2020-07-01') # closing schools (end calibration wave1)

    if t <= t1 + tau_days:
        return contact_matrix(t, df_google, Nc_all, school=1)
    elif t1 + tau_days < t <= t1 + tau_days + l_days:
        policy_old = contact_matrix(t, df_google, Nc_all, school=1)
        policy_new = contact_matrix(t, df_google, Nc_all, prev_home, prev_schools, prev_work, prev_transport, 
                                    prev_leisure, prev_others, school=0)
        return ramp_fun(policy_old, policy_new, t, tau_days, l, t1)
    elif t1 + tau_days + l_days < t <= t2:
        return contact_matrix(t, df_google, Nc_all, prev_home, prev_schools, prev_work, prev_transport, 
                              prev_leisure, prev_others, school=0)
    elif t2 < t <= t3:
        return contact_matrix(t, df_google, Nc_all, prev_home, prev_schools, prev_work, prev_transport, 
                              prev_leisure, prev_others, school=0.15)
    elif t3 < t <= t4:
        return contact_matrix(t, df_google, Nc_all, prev_home, prev_schools, prev_work, prev_transport, 
                              prev_leisure, prev_others, school=0.65)
    else:
        return contact_matrix(t, df_google, Nc_all, prev_home, prev_schools, prev_work, prev_transport, 
                              prev_leisure, prev_others, school=0)


In [6]:
#### Load the model parameters using `get_COVID19_SEIRD_parameters()`.
params = model_parameters.get_COVID19_SEIRD_parameters()

params.update({'df_google': df_google,
              'Nc_all' : Nc_all,
               'l' : 5,
               'tau' : 5,
               'prev_schools': 0.5,
               'prev_work': 0.5,
               'prev_transport': 0.5,
               'prev_leisure': 0.5,
               'prev_others': 0.5,
               'prev_home' : 0.5
              })

# Define the initial condition: one exposed inidividual in every age category
initial_states = {'S': initN, 'E': np.ones(levels)}
# Initialize the model
model = models.COVID19_SEIRD(initial_states, params, time_dependent_parameters={'Nc': wave1_policies})

## Perform calibration

In [8]:
# Date of first data collection
start_calibration = '2020-03-15'
# Last datapoint used to calibrate basic reproduction number
end_calibration = '2020-07-01'

# Path where figures should be stored
fig_path = '../../results/calibrations/COVID19_SEIRD/national/'
# Path where MCMC samples should be saved
samples_path = '../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/'
# Spatial unit: Belgium
spatial_unit = 'BE_6_prev_thin'

In [ ]:
init_warmup = 30
maxiter = 100
popsize = 100
processes = 16 # voor eraser!! ## PROCESSES=1 to debug!
steps_mcmc = 3000#3000
discard = 1000#1000

# define dataset
data=[df_sciensano['H_in'][start_calibration:end_calibration]]
states = [["H_in"]]

####################################################
####### CALIBRATING BETA AND COMPLIANCE RAMP #######
####################################################

print('------------------------------------')
print('CALIBRATING BETA AND COMPLIANCE RAMP')
print('------------------------------------\n')
print('Using data from '+start_calibration+' until '+end_calibration+'\n')
print('1) Particle swarm optimization\n')

# set PSO optimisation settings
parNames = ['sigma_data','beta','l','tau',
            'prev_schools', 'prev_work', 'prev_transport', 'prev_leisure', 'prev_others', 'prev_home', 'warmup']
bounds=((1,2000),(0.010,0.060),(0.1,20),(0.1,20),
        (0,1),(0,1),(0,1),(0,1),(0,1),(0,1), (30,60))

# run PSO optimisation
theta_pso = pso.fit_pso(model,data,parNames,states,bounds,maxiter=maxiter,popsize=popsize,
                    start_date=start_calibration,warmup=init_warmup, processes=processes) 
warmup = int(theta_pso[-1])

# run MCMC sampler
print('\n2) Markov-Chain Monte-Carlo sampling\n')
parNames_mcmc = parNames[:-1]
bounds_mcmc=((1,2000),(0.020,0.060),(0.001,20),(0.001,20),
             (0,1),(0,1),(0,1),(0,1),(0,1),(0,1))

theta = theta_pso[:-1]
ndim = len(theta)
nwalkers = ndim*2
perturbations = ([1]+(ndim-1)*[1e-3]) * np.random.randn(nwalkers, ndim)
pos = theta + perturbations

sampler = emcee.EnsembleSampler(nwalkers, ndim, objective_fcns.log_probability,
                    args=(model, bounds_mcmc, data, states, parNames_mcmc, None, start_calibration, warmup))
sampler.run_mcmc(pos, steps_mcmc, progress=True)

thin = 0
try:
    autocorr = sampler.get_autocorr_time()
    thin = int(0.5 * np.min(autocorr))
except:
    print('Warning: The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s).\nUse this estimate with caution and run a longer chain!')

from covid19model.optimization.run_optimization import checkplots

checkplots(sampler, discard, thin, fig_path, spatial_unit, figname='BETA_RAMP_GOOGLE_WAVE1', 
           labels=['$\sigma_{data}$','$\\beta$','l','$\\tau$',
                   'prev_schools', 'prev_work', 'prev_transport', 'prev_leisure', 'prev_others', 'prev_home'])

#############################################
####### Output to dictionary ################
#############################################

flat_samples = sampler.get_chain(discard=discard,thin=thin,flat=True)

samples_dict_wave1 = {}
for count,name in enumerate(parNames_mcmc):
    samples_dict_wave1[name] = flat_samples[:,count].tolist()

samples_dict_wave1.update({
    'theta_pso' : list(theta_pso),
    'warmup' : warmup,
    'calibration_data' : states[0][0],
    'start_date' : start_calibration,
    'end_date' : end_calibration,
    'maxiter' : maxiter,
    'popsize': popsize,
    'steps_mcmc': steps_mcmc,
    'discard' : discard,
})

with open(samples_path+str(spatial_unit)+'_'+str(datetime.date.today())+'_WAVE1_GOOGLE.json', 'w') as fp:
    json.dump(samples_dict_wave1, fp)

------------------------------------
CALIBRATING BETA AND COMPLIANCE RAMP
------------------------------------

Using data from 2020-03-15 until 2020-07-01

1) Particle swarm optimization

No constraints given.
New best for swarm at iteration 1: [2.65289788e+02 2.69473058e-02 5.37532374e+00 1.00000000e-01
 4.56774435e-01 0.00000000e+00 1.00000000e+00 8.37298794e-01
 5.13722791e-01 1.00000000e+00 5.30626002e+01] 651.1880886005232
Best after iteration 1: [2.65289788e+02 2.69473058e-02 5.37532374e+00 1.00000000e-01
 4.56774435e-01 0.00000000e+00 1.00000000e+00 8.37298794e-01
 5.13722791e-01 1.00000000e+00 5.30626002e+01] 651.1880886005232
New best for swarm at iteration 2: [2.28996455e+02 3.50345375e-02 1.00000000e-01 7.03733590e+00
 4.49056491e-01 0.00000000e+00 4.98273078e-01 3.30448221e-01
 5.60061175e-01 0.00000000e+00 3.28667112e+01] 650.5547626353508
Best after iteration 2: [2.28996455e+02 3.50345375e-02 1.00000000e-01 7.03733590e+00
 4.49056491e-01 0.00000000e+00 4.98273078e-01 3.3

New best for swarm at iteration 26: [3.89004200e+01 4.35813052e-02 1.00000000e-01 1.13116076e+01
 0.00000000e+00 0.00000000e+00 5.12473010e-01 5.88248019e-01
 5.67998711e-01 1.48066745e-01 4.62609759e+01] 455.4185664078193
Best after iteration 26: [3.89004200e+01 4.35813052e-02 1.00000000e-01 1.13116076e+01
 0.00000000e+00 0.00000000e+00 5.12473010e-01 5.88248019e-01
 5.67998711e-01 1.48066745e-01 4.62609759e+01] 455.4185664078193
Best after iteration 27: [3.89004200e+01 4.35813052e-02 1.00000000e-01 1.13116076e+01
 0.00000000e+00 0.00000000e+00 5.12473010e-01 5.88248019e-01
 5.67998711e-01 1.48066745e-01 4.62609759e+01] 455.4185664078193
Best after iteration 28: [3.89004200e+01 4.35813052e-02 1.00000000e-01 1.13116076e+01
 0.00000000e+00 0.00000000e+00 5.12473010e-01 5.88248019e-01
 5.67998711e-01 1.48066745e-01 4.62609759e+01] 455.4185664078193
Best after iteration 29: [3.89004200e+01 4.35813052e-02 1.00000000e-01 1.13116076e+01
 0.00000000e+00 0.00000000e+00 5.12473010e-01 5.8824801

Best after iteration 55: [3.67148769e+01 4.38343665e-02 1.00000000e-01 9.37239561e+00
 0.00000000e+00 0.00000000e+00 5.27979811e-01 7.09442816e-01
 6.80918669e-01 4.28426553e-02 4.64468878e+01] 445.75508763317896
Best after iteration 56: [3.67148769e+01 4.38343665e-02 1.00000000e-01 9.37239561e+00
 0.00000000e+00 0.00000000e+00 5.27979811e-01 7.09442816e-01
 6.80918669e-01 4.28426553e-02 4.64468878e+01] 445.75508763317896
Best after iteration 57: [3.67148769e+01 4.38343665e-02 1.00000000e-01 9.37239561e+00
 0.00000000e+00 0.00000000e+00 5.27979811e-01 7.09442816e-01
 6.80918669e-01 4.28426553e-02 4.64468878e+01] 445.75508763317896
Best after iteration 58: [3.67148769e+01 4.38343665e-02 1.00000000e-01 9.37239561e+00
 0.00000000e+00 0.00000000e+00 5.27979811e-01 7.09442816e-01
 6.80918669e-01 4.28426553e-02 4.64468878e+01] 445.75508763317896
Best after iteration 59: [3.67148769e+01 4.38343665e-02 1.00000000e-01 9.37239561e+00
 0.00000000e+00 0.00000000e+00 5.27979811e-01 7.09442816e-01
 

Best after iteration 84: [3.62347134e+01 4.38303404e-02 1.00000000e-01 9.43321830e+00
 0.00000000e+00 0.00000000e+00 5.32671535e-01 7.11024326e-01
 6.81158350e-01 4.44319286e-02 4.64037597e+01] 445.7307821903501
New best for swarm at iteration 85: [3.62375606e+01 4.38301096e-02 1.00000000e-01 9.43353440e+00
 0.00000000e+00 0.00000000e+00 5.32562525e-01 7.10935263e-01
 6.81158648e-01 4.43548334e-02 4.64112482e+01] 445.7307256693293
Best after iteration 85: [3.62375606e+01 4.38301096e-02 1.00000000e-01 9.43353440e+00
 0.00000000e+00 0.00000000e+00 5.32562525e-01 7.10935263e-01
 6.81158648e-01 4.43548334e-02 4.64112482e+01] 445.7307256693293
New best for swarm at iteration 86: [3.61978278e+01 4.38296552e-02 1.00000000e-01 9.43918160e+00
 0.00000000e+00 0.00000000e+00 5.33834784e-01 7.10966514e-01
 6.81309169e-01 4.44471029e-02 4.64078646e+01] 445.7305552654074
Best after iteration 86: [3.61978278e+01 4.38296552e-02 1.00000000e-01 9.43918160e+00
 0.00000000e+00 0.00000000e+00 5.33834784e-0

  0%|          | 0/3000 [00:00<?, ?it/s]/home/data/jvergeyn/.conda/envs/COVID_MODEL/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]
  2%|▏         | 72/3000 [13:20<12:16:28, 15.09s/it]

In [ ]:
end_sim = '2020-07-01'

fig,ax=plt.subplots(figsize=(10,4))
for i in range(100):
    # Sampling
    idx, model.parameters['beta'] = random.choice(list(enumerate(samples_dict_wave1['beta'])))
    model.parameters['l'] = samples_dict_wave1['l'][idx] 
    model.parameters['tau'] = samples_dict_wave1['tau'][idx]    
    model.parameters['prev_schools'] = samples_dict_wave1['prev_schools'][idx]    
    model.parameters['prev_work'] = samples_dict_wave1['prev_work'][idx]     
    model.parameters['prev_transport'] = samples_dict_wave1['prev_transport'][idx]    
    model.parameters['prev_leisure'] = samples_dict_wave1['prev_leisure'][idx]     
    model.parameters['prev_others'] = samples_dict_wave1['prev_others'][idx]      

    # Simulate
    y_model = model.sim(end_sim,start_date=start_calibration,warmup=theta_pso[-1])
    # Plot
    ax.plot(y_model['time'],y_model["H_in"].sum(dim="Nc"),color='blue',alpha=0.1)

ax.scatter(df_sciensano[start_calibration:end_calibration].index,df_sciensano['H_in'][start_calibration:end_calibration],color='black',alpha=0.6,linestyle='None',facecolors='none')
ax = _apply_tick_locator(ax)
ax.set_xlim('2020-02-15',end_sim)
fig.savefig(fig_path+'others/FIT_WAVE1_GOOGLE_'+spatial_unit+'_'+str(datetime.date.today())+'.pdf', dpi=400, bbox_inches='tight')

## Visualizations

### Goodness-of-fit: $\beta$, $l$ and $\tau$

In [ ]:
old_theta = {name:par for name,par in zip(parNames, theta) } 
old_theta

In [ ]:
params.update({'samples_dict': samples_dict})

# Re-initialize the model
model = models.COVID19_SEIRD(initial_states, params, time_dependent_parameters={'Nc': google_lockdown, 'beta': switch_beta})

In [ ]:
end_sim = end_calibration

fig,ax=plt.subplots(figsize=(10,4))
for i in range(10):
    # Sampling
    idx, model.parameters['beta'] = random.choice(list(enumerate(samples_dict_wave1['beta'])))
    model.parameters['l'] = samples_dict_wave1['l'][idx] 
    model.parameters['tau'] = samples_dict_wave1['tau'][idx]    
    model.parameters['prev_schools'] = samples_dict_wave1['prev_schools'][idx]    
    model.parameters['prev_work'] = samples_dict_wave1['prev_work'][idx]     
    model.parameters['prev_transport'] = samples_dict_wave1['prev_transport'][idx]    
    model.parameters['prev_leisure'] = samples_dict_wave1['prev_leisure'][idx]     
    model.parameters['prev_others'] = samples_dict_wave1['prev_others'][idx]      

    # Simulate
    y_model = model.sim(end_sim,start_date=start_calibration,warmup=51)
    # Plot
    ax.plot(y_model['time'],y_model["H_in"].sum(dim="Nc"),color='blue',alpha=0.1)

ax.scatter(df_sciensano[start_calibration:end_calibration].index,df_sciensano['H_in'][start_calibration:end_calibration],color='black',alpha=0.6,linestyle='None',facecolors='none')
ax = _apply_tick_locator(ax)
ax.set_xlim('2020-03-15',end_sim)
#plt.savefig(fig_path+'others/FIT_WAVE1_GOOGLE_'+spatial_unit+'_'+str(datetime.date.today())+'.pdf', dpi=400, bbox_inches='tight')

### $R_0$

In [ ]:
fig,ax=plt.subplots(figsize=(10,4))
ax.hist(samples_dict['R0'],bins=30)
plt.savefig(fig_path+'others/R0_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

### Seroprelevance

In [ ]:
end_sim = '2020-06-01'

fig,ax=plt.subplots(figsize=(10,4))
for i in range(10):
    model.parameters = draw_sample_COVID19_SEIRD_google(model.parameters,samples_dict)
    y_model = model.sim(end_sim,start_date=start_data,excess_time=samples_dict['warmup'])
    ax.plot((1-y_model["S"].sel(time=end_sim)/y_model["S"].isel(time=0))*100,alpha=0.03,color='blue',linewidth=2)

ax.set_xlabel('age')
ax.set_ylabel("seroprelevance (%)" )
ax.set_ylim()
plt.xticks(np.arange(9), ('[0-10[', '[10-20[', '[20-30[', '[30-40[', '[40-50[', '[50-60[', '[60-70[', '[70-80[', '[80-$\infty$['))

plt.savefig(fig_path+'others/SEROPRELEVANCE_'+end_sim+'_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

### Lockdown release

In [ ]:
end_sim = '2020-09-01'
n=100
percentile = 0.99

y_model = model.sim(end_sim,start_date=start_data,excess_time=samples_dict['warmup'],N=n,draw_fcn=draw_sample_COVID19_SEIRD_google,samples=samples_dict)

fig,ax = plt.subplots(figsize=(10,4))
ax.fill_between(pd.to_datetime(y_model['time'].values),y_model["H_tot"].quantile(1-percentile,dim="draws").sum(dim="Nc"), y_model["H_tot"].quantile(percentile,dim="draws").sum(dim="Nc"),alpha=0.30)
ax.plot(y_model['time'],y_model["H_tot"].mean(dim="draws").sum(dim="Nc"),'--')
ax.scatter(df_sciensano[:end_sim].index,df_sciensano['H_tot'][:end_sim],color='black',alpha=0.4,linestyle='None',facecolors='none')
ax = _apply_tick_locator(ax)
plt.savefig(fig_path+'others/LOCKDOWN_RELEASE_WAVE1_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

### Save results to a .csv

In [ ]:
index = y_model['time'].values
columns = ['WAVE1_INCIDENCES_MEAN','WAVE1_INCIDENCES_MEDIAN','WAVE1_INCIDENCES_LL','WAVE1_INCIDENCES_UL']
data = np.transpose(np.array([
        y_model["H_tot"].mean(dim="draws").sum(dim="Nc").values.tolist(),
        y_model["H_tot"].median(dim="draws").sum(dim="Nc").values.tolist(),
        y_model["H_tot"].quantile(1-percentile,dim="draws").sum(dim="Nc").values.tolist(),
        y_model["H_tot"].quantile(percentile,dim="draws").sum(dim="Nc").values.tolist()
       ]))

In [ ]:
df = pd.DataFrame(data,columns=columns,index=index)
df.head()

In [ ]:
df.to_csv('../../results/predictions/FAGG/COVID19_SEIRD/WAVE1.csv',index=True)